# Select and distribute additional Arousal articles for phrase annotation

## Goal is deeper annotation.
- Doubling or more our previous annotation for arousal.
- Phrase based annotation so we can be most flexible and assess at higher levels.

## Secondary goal, more cross-validation.
- 1/2 of new articles will be annotated by all.
- 1/2 of new articles will be annotated by only 1 annotator.

In [ ]:
dest_dir = '/Users/ccarey/Documents/Projects/NAMI/rdoc/pdfs/20160122_rdoc_project'
%mkdir {dest_dir}
%cd {dest_dir}

### Get a list of all our previous used pubmed ids.

In [ ]:
from __future__ import print_function
import glob
import os
import re
prev_annotated_pdfs_dir = '/Users/ccarey/Documents/Projects/NAMI/rdoc/pdfs/all_pdfs_annotated_pmid_names/*.pdf'
pdfs = glob.glob(prev_annotated_pdfs_dir)

In [ ]:
pdfs = [os.path.basename(pdf) for pdf in pdfs]

#### Crosscheck article counts in previous annotations.
Should find this many minus the 3 marked as 'NA' i.e. 3 documents did not have a pubmed id.
(171 - 3) = 168

In [ ]:
%ls $prev_annotated_pdfs_dir | wc -l

In [ ]:
pattern = '([0-9]{8,8})'
p = re.compile(pattern)
tot = len([p.search(pdf).group() for pdf in pdfs if p.search(pdf)])
pmids = set([p.search(pdf).group() for pdf in pdfs if p.search(pdf)])
pmids = list(pmids)
pmids.sort()
uniq = len(pmids)
# print('So... {} pubmed_ids were found from total of {} previously annotated '
#       'or marked irrelevant / misclassified'.format(uniq, tot))
# print(pmids)

So... 73 pubmed_ids were found from total of 168 previously annotated or marked irrelevant / misclassified

['20685988', '20695690', '20815182', '21319926', '21531705', '21613467', '21699821', '21849230', '21957257', '22379238', '22379245', '22438994', '22447249', '22575329', '23074247', '23083918', '23088207', '23143607', '23452958', '23503620', '23558179', '23622762', '23646134', '23647728', '23709163', '23744445', '23904684', '23928891', '23954763', '23957953', '24023823', '24045586', '24101292', '24116095', '24231418', '24285346', '24293773', '24333377', '24333745', '24359877', '24376698', '24388670', '24470693', '24511281', '24725811', '24740391', '24770625', '24804717', '24806675', '24870123', '24933724', '24980898', '25017671', '25036160', '25126029', '25126038', '25136085', '25142564', '25154749', '25160677', '25197810', '25258728', '25261920', '25280468', '25348131', '25734385', '25740534', '25773639', '25774613', '25788679', '25834059', '25898427', '25913552']

#### Crosscheck articles counts in existing medic database.
May be necessary to restart postgres server.

In [ ]:
# !pg_ctl -D /usr/local/var/postgres -l /usr/local/var/postgres/server.log start
medic_pmids = !medic --format tsv write ALL 2>/dev/null | cut  -f 1 | uniq | sort
print(set(pmids).difference(set(in_medic)))
print(set(medic_pmids).difference(set(pmids)))

From previous notes in 'fetch_pdfs_batch3_2015.10.12.py':
  - \#\# caught output as this, but seems incorrect as it had AR04 '23941878' instead of '25258728'

Article 23941878 is an arousal document that did not go out in previous rounds due to a mislabeling issue. (25258728 did go out and we corrected the naming)

In [ ]:
prev_pmids = pmids
new_pmids = ['23941878']

In [ ]:
#from __future__ import print_function
from Bio import Entrez
from subprocess import check_call
from shutil import copy2
#import glob
import time
import imp
import os
url2p = imp.load_source('Url2PubmedPmcPdf', '/Users/ccarey/Documents/Projects/NAMI/rdoc/scripts/Url2PubmedPmcPdf.py')
Entrez.email = "charlieccarey@gmail.com"

def narrow_id_list(found_ids, omit_ids):
    found_but_omit = list(set(found_ids) & set(omit_ids))
    found_and_keep = list(set(found_ids) - set(omit_ids))
    print('Removed this many ids: {}'.format(len(found_but_omit)))
    return(found_and_keep)

def pubmed_central_search_to_pubmed_id(search_string, retmax=20):
    # verify how many records match
    handle = Entrez.egquery(term=search_string)
    record = Entrez.read(handle)
    # maybe useful if we are dealing with 100s of ids and don't want to overwhelm server?
    for row in record["eGQueryResult"]:
        if row["DbName"] == "pubmed":
            print(row["Count"])
    # fetch the ids for those records
    handle = Entrez.esearch(db="pubmed", retmax=retmax, term=search_string)
    record = Entrez.read(handle)
    pubmed_ids = record["IdList"]
    return(pubmed_ids)

def fetch_pdfs(pubmed_ids, stub_name):
    u = url2p.Url2PubmedPmcPdf(pubmed_ids)
    urls = u.get_urls()
    found = []
    for url in urls:
        if url['url'] is not None:
            cmd = 'curl -L {} -o {}.pdf'.format(url['url'], stub_name + url['pubmed'])
            # print(cmd)
            check_call(cmd, shell = True)
            time.sleep(10)
            found.append(url['pubmed'])
    return(found)

# def copy_pdf_append_initials(initials):
#     pdfs = glob.glob('*.pdf')
#     for i in initials:
#         os.mkdir(i)
#         for p in pdfs:
#             pi = p.replace('.pdf', '_' + i + '.pdf')
#             copy2(p, os.path.join(i, pi))

def search_and_summarize(search_name, query, omit_ids):
    ids = pubmed_central_search_to_pubmed_id(query, retmax=1000000)
    new_ids = narrow_id_list(ids, omit_ids)
    print('{} search of pubmed found {} ids of which {} are new'.format(search_name, len(ids), len(new_ids)))
    return(new_ids)

## Might be limiting our results too much by using pubmed. Pubmed central seems to have more free full text results.

For example, modifying the searches slightly for pubmed central vs. pubmed.

AR00_pmc = ("arousal"[MeSH Terms] OR "arousal"[All Fields]) AND ("free full text"[sb] AND "2010/06/24"[PubDate] : "2015/06/22"[PubDate] AND "humans"[MeSH Terms])

AR00_pubmed ='("arousal"[MeSH Terms] OR "arousal"[All Fields]) AND ("loattrfree full text"[sb] AND "2010/06/24"[PDat] : "2015/06/22"[PDat] AND "humans"[MeSH Terms])'

pmc :
- 13656 
- some are sponsored in readable format but have to click through to publisher to get pdf.

pubmed : 
- 6503 
- all pdf.

### Desire to send out more articles. 30 in common to all, 30 unique to all
 - 30 common to all + 30 unique per annotator * 4 annotators
 - 150 articles total desired.

In [ ]:
AR00 = '("arousal"[MeSH Terms] OR "arousal"[All Fields]) AND ("loattrfree full text"[sb] AND "2010/06/24"[PDat] : "2015/06/22"[PDat] AND "humans"[MeSH Terms])'
AR01 = '"arousal"[All Fields] AND (("arousal"[MeSH Terms] OR "arousal"[All Fields]) AND ("self report"[MeSH Terms] OR ("self"[All Fields] AND "report"[All Fields]) OR "self report"[All Fields]) AND ("weights and measures"[MeSH Terms] OR ("weights"[All Fields] AND "measures"[All Fields]) OR "weights and measures"[All Fields] OR "scales"[All Fields])) AND ("loattrfree full text"[sb] AND "2010/06/24"[PDat] : "2016/01/22"[PDat] AND "humans"[MeSH Terms])'
AR02 = '"arousal"[All Fields] AND (("self-assessment"[MeSH Terms] OR "self-assessment"[All Fields] OR ("self"[All Fields] AND "assessment"[All Fields]) OR "self assessment"[All Fields]) AND ("manikins"[MeSH Terms] OR "manikins"[All Fields] OR "mannequin"[All Fields])) AND ("loattrfree full text"[sb] AND "2010/06/24"[PDat] : "2016/01/22"[PDat] AND "humans"[MeSH Terms])'
AR03 = '"arousal"[All Fields] AND "EEG"[All Fields] AND ("loattrfree full text"[sb] AND "2010/06/24"[PDat] : "2016/01/22"[PDat] AND "humans"[MeSH Terms])'
AR04 = '"arousal"[All Fields] AND "psychomotor vigilance"[All Fields] AND ("loattrfree full text"[sb] AND "2010/06/24"[PDat] : "2016/01/22"[PDat] AND "humans"[MeSH Terms])'
AR05 = '"arousal"[All Fields] AND "startle"[All Fields] AND ("loattrfree full text"[sb] AND "2010/06/24"[PDat] : "2016/01/22"[PDat] AND "humans"[MeSH Terms])'
AR05_plus = '"arousal"[All Fields] AND "startle"[All Fields] AND ("loattrfree full text"[sb] AND "2010/06/24"[PDat] : "2016/01/22"[PDat] AND ("humans"[MeSH Terms] OR "humans"[All Fields] OR "human"[All Fields]))'

# AR00: there would be a lot. Use if not enough of others.
# AR01: some used already. USE.
# AR02: 0 found in original search, don't use.
# AR03: some used already. USE.
# AR04: some used already. USE.
# AR05: some used already. USE.

# already sent out for review under batches 2 or 3 (labels 02_ or 03_): prev_pmids

AR00_ids = search_and_summarize(search_name='AR00', query=AR00, omit_ids=prev_pmids)
print()
AR01_ids = search_and_summarize(search_name='AR01', query=AR01, omit_ids=prev_pmids)
print()
AR02_ids = search_and_summarize(search_name='AR02', query=AR02, omit_ids=prev_pmids)
print()
AR03_ids = search_and_summarize(search_name='AR03', query=AR03, omit_ids=prev_pmids)
print()
AR04_ids = search_and_summarize(search_name='AR04', query=AR04, omit_ids=prev_pmids)
print()
AR05_ids = search_and_summarize(search_name='AR05', query=AR05, omit_ids=prev_pmids)
print()

In [ ]:
import collections

# total found across all searches
ids = [AR01_ids, AR02_ids, AR03_ids, AR04_ids, AR05_ids]
ids = [j for i in ids for j in i]
print('total new pmids found across all searches : {}'.format(len(ids)))

ids = [AR01_ids, AR02_ids, AR03_ids, AR04_ids, AR05_ids]
ids = [j for i in ids for j in i]
dups = [item for item, count in collections.Counter(ids).items() if count > 1]
print('duplicate pmids found across all searches : {}'.format(dups))

ids = [AR01_ids, AR02_ids, AR03_ids, AR04_ids, AR05_ids]
ids = [set(id_list) for id_list in ids]
ids = set.union(*ids)
print('unique new pmids found across all searches : {}'.format(len(ids)))

### Found nearly 150 new articles, add a few more from general search.

- total new pmids found across all searches : 144
- duplicate pmids found across all searches : ['20408942']
- unique new pmids found across all searches : 143

### Find 7 additional ids to get total 150 new ids.

In [340]:
AR00_specific_ids = narrow_id_list(AR00_ids, omit_ids=ids)

Removed this many ids: 143


### Fetch the pdfs for each search term.

Note, we should be here: '/Users/ccarey/Documents/Projects/NAMI/rdoc/pdfs/20160122_rdoc_project'

In [ ]:
%pwd

In [ ]:
# 4th batch, Arousal, subterm...
# Note: There were no ids for AR02.

batch = '04'

fetch_pdfs(AR01_ids, batch + '_AR01_')
fetch_pdfs(AR03_ids, batch + '_AR03_')
fetch_pdfs(AR04_ids, batch + '_AR04_')
fetch_pdfs(AR05_ids, batch + '_AR05_')

In [ ]:
# Extend to a few more generic results to round out
# to ~150 and account for some that will be missing.
fetch_pdfs(AR00_specific_ids[0:40], batch + '_AR00_')

In [ ]:
all_found = glob.glob( './*.pdf')
len(all_found)

In [ ]:
import collections

def report_found(search_ids, glob_pattern, id_pattern):
    '''Report the pmids found within the search set of files.'''
    files = glob.glob(glob_pattern)
    p = re.compile('[0-9]{8,8}')
    found = [p.search(pdf).group() for pdf in files if p.search(pdf)]
    not_found = set(search_ids).difference(found)
    print()
    #print('{} searched = {}'.format(len(search_ids), search_ids))
    #print('{} found = {}'.format(len(found), found))
    print('{} {} searched.'.format(len(search_ids), glob_pattern))
    print('{} {} found.'.format(len(found), glob_pattern))
    print('{} {} not_found = {}'.format(len(not_found), glob_pattern, not_found))
    return(found)

all_found_d = {}
all_found_d['AR00'] = report_found(AR00_specific_ids[0:40], glob_pattern='04_AR00_*', id_pattern='[0-9]{8,8}')
all_found_d['AR01'] = report_found(AR01_ids, glob_pattern='04_AR01_*', id_pattern='[0-9]{8,8}')
all_found_d['AR03'] = report_found(AR03_ids, glob_pattern='04_AR03_*', id_pattern='[0-9]{8,8}')
all_found_d['AR04'] = report_found(AR04_ids, glob_pattern='04_AR04_*', id_pattern='[0-9]{8,8}')
all_found_d['AR05'] = report_found(AR05_ids, glob_pattern='04_AR05_*', id_pattern='[0-9]{8,8}')

print()
print('------- Reporting Found: ----------')
print()
print('{} Total found'.format(sum([len(v) for v in all_found_d.values()])))
print()
all_found_d = collections.OrderedDict(sorted(all_found_d.items()))
for k,v in all_found_d.iteritems():
    print('{}: {}'.format(k, v))

### Reported found or not found for batch 4
    40 04_AR00_* searched.
    32 04_AR00_* found.
    8 04_AR00_* not_found = set(['22715192', '25109588', '22608701', '21304250', '22998863', '21697711', '26035194', '24330893'])

    2 04_AR01_* searched.
    1 04_AR01_* found.
    1 04_AR01_* not_found = set(['22044631'])

    99 04_AR03_* searched.
    86 04_AR03_* found.
    13 04_AR03_* not_found = set(['24718047', '24920020', '25972166', '24552365', '20729099', '23440753', '22068747', '25794203', '24177246', '20408942', '24169072', '24469936', '21051491'])

    17 04_AR04_* searched.
    12 04_AR04_* found.
    5 04_AR04_* not_found = set(['21564364', '20408942', '24905752', '22436093', '23171222'])

    26 04_AR05_* searched.
    25 04_AR05_* found.
    1 04_AR05_* not_found = set(['22387928'])

    ------- Reporting Found: ----------
    
    156 Total found

    AR00: ['20410871', '20532489', '21050743', '21278378', '21278907', '22004270', '22048839', '22146934', '22169884', '22233352', '22378876', '22414937', '22474609', '22505867', '22686386', '22715197', '22805501', '22832959', '22968207', '23190433', '23299717', '23504052', '23555220', '23770566', '23934417', '24126129', '24466064', '24512610', '24651580', '24933663', '24964082', '25521352']
    AR01: ['22664396']
    AR03: ['20450941', '20525011', '20584721', '20615239', '20620104', '20634711', '20663220', '20842164', '20857862', '21077571', '21118712', '21120131', '21179552', '21203376', '21206465', '21276977', '21280045', '21335015', '21397252', '21419826', '21426626', '21675365', '21731598', '21816115', '21854953', '21886801', '21909371', '21954087', '22043127', '22119526', '22131608', '22163262', '22215928', '22314045', '22377810', '22379239', '22467988', '22496862', '22665872', '22986355', '22998925', '23055094', '23060019', '23127585', '23193115', '23326604', '23643925', '23707592', '23731439', '23786695', '23810448', '23899724', '23929944', '23940642', '23990240', '23997704', '24015304', '24125792', '24127147', '24214921', '24235891', '24252875', '24259275', '24260331', '24266644', '24412227', '24426818', '24453310', '24457211', '24505292', '24529045', '24571111', '24618591', '24705497', '24899756', '24930577', '25024660', '25051268', '25061837', '25348125', '25424865', '25456277', '25581922', '25756280', '25759762', '25883640']
    AR04: ['20955866', '21677894', '21826029', '22035386', '22215925', '22239924', '22294809', '22470524', '22959616', '23941878', '25142762', '25325584']
    AR05: ['20049632', '20497902', '20661292', '20939652', '21034683', '21259270', '21376761', '21392554', '21440905', '21463060', '21477924', '21550590', '21623612', '21626350', '21898707', '22088577', '22285891', '22286850', '22315106', '22776995', '22911829', '24156344', '25036222', '25107317', '25749431']

#### 156 pubmed ids were uniquely found and downloaded successfully.
Randomly choose 32 to assign to everyone. And from the remaining 124 (=156-31) assign 31 different ones to each of 4 people.

Note: '23941878' which we missed previously is now covered in set AR04.

In [393]:
import random
files = glob.glob('04_AR*.pdf')
idxs = range(len(files))
random.shuffle(idxs)

everyone = [files[idx] for idx in idxs[0:32]]
mk = [files[idx] for idx in idxs[32:63]]
jl = [files[idx] for idx in idxs[63:94]]
tc = [files[idx] for idx in idxs[94:125]]
cc = [files[idx] for idx in idxs[125:156]]

In [414]:
def copy_to_annotator(files, annotator, suffix):
    %mkdir {annotator}
    %mkdir {annotator}/annotated
    %mkdir {annotator}/irrelevant
    for f in files:
        f_dest = f.split('.')[0] + suffix
        %cp {f} ./{annotator}/{f_dest}

copy_to_annotator(mk, 'mk', '_mk.pdf')
copy_to_annotator(everyone, 'mk', '_mk.pdf')

copy_to_annotator(jl, 'jl', '_jl.pdf')
copy_to_annotator(everyone, 'jl', '_jl.pdf')

copy_to_annotator(mk, 'tc', '_tc.pdf')
copy_to_annotator(everyone, 'tc', '_tc.pdf')

copy_to_annotator(mk, 'cc', '_cc.pdf')
copy_to_annotator(everyone, 'cc', '_cc.pdf')


mkdir: mk: File exists
mkdir: mk: File exists
mkdir: mk/annotated: File exists
mkdir: mk/irrelevant: File exists
mkdir: jl: File exists
mkdir: jl: File exists
mkdir: jl/annotated: File exists
mkdir: jl/irrelevant: File exists
mkdir: tc: File exists
mkdir: tc: File exists
mkdir: tc/annotated: File exists
mkdir: tc/irrelevant: File exists
mkdir: cc: File exists
mkdir: cc: File exists
mkdir: cc/annotated: File exists
mkdir: cc/irrelevant: File exists


In [ ]:
for f in mk:
    f_dest = f.split('.')[0] + '_mk.pdf'
    %cp {f} ./mk/{f_dest}

for f in jl:
    f_dest = f.split('.')[0] + '_jl.pdf'
    %cp {f} ./jl/{f_dest}
    
for f in tc:
    f_dest = f.split('.')[0] + '_tc.pdf'
    %cp {f} ./tc/{f_dest}

for f in cc:
    f_dest = f.split('.')[0] + '_cc.pdf'
    %cp {f} ./tc/{f_dest}

## Add the new abstracts to our medic database
Insert fails if any are already in db.
Update will overwrite previous records.

#### Previous set for our records:

In [25]:
%cd /Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/task_data_temp
prev = !medic --format tsv write ALL 2> /dev/null | cut -f 1
print(prev)

/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/task_data_temp
['20695690', '21699821', '21849230', '22379238', '22438994', '23074247', '23088207', '23143607', '23452958', '23622762', '23709163', '23904684', '23941878', '24725811', '24770625', '24806675', '24980898', '25017671', '25258728', '25348131', '23928891', '23744445', '25774613', '25773639', '25740534', '24870123', '24740391', '21957257', '21613467', '22447249', '21319926', '23503620', '21531705', '24023823', '24231418', '23558179', '25734385', '23647728', '23083918', '22575329', '22379245', '25197810', '25136085', '25126038', '25142564', '25126029', '24470693', '24333377', '24293773', '24116095', '24101292', '24045586', '23954763', '23646134', '20815182', '25898427', '24285346', '25036160', '24511281', '23957953', '25280468', '24388670', '25154749', '24933724', '24359877', '24333745', '24804717', '25834059', '25788679', '25261920', '25160677', '25913552', '24376698', '20685988']


In [17]:
all_found_d = {'AR00': ['20410871', '20532489', '21050743', '21278378', '21278907', '22004270', '22048839', '22146934', '22169884', '22233352', '22378876', '22414937', '22474609', '22505867', '22686386', '22715197', '22805501', '22832959', '22968207', '23190433', '23299717', '23504052', '23555220', '23770566', '23934417', '24126129', '24466064', '24512610', '24651580', '24933663', '24964082', '25521352'],
'AR01': ['22664396'],
'AR03': ['20450941', '20525011', '20584721', '20615239', '20620104', '20634711', '20663220', '20842164', '20857862', '21077571', '21118712', '21120131', '21179552', '21203376', '21206465', '21276977', '21280045', '21335015', '21397252', '21419826', '21426626', '21675365', '21731598', '21816115', '21854953', '21886801', '21909371', '21954087', '22043127', '22119526', '22131608', '22163262', '22215928', '22314045', '22377810', '22379239', '22467988', '22496862', '22665872', '22986355', '22998925', '23055094', '23060019', '23127585', '23193115', '23326604', '23643925', '23707592', '23731439', '23786695', '23810448', '23899724', '23929944', '23940642', '23990240', '23997704', '24015304', '24125792', '24127147', '24214921', '24235891', '24252875', '24259275', '24260331', '24266644', '24412227', '24426818', '24453310', '24457211', '24505292', '24529045', '24571111', '24618591', '24705497', '24899756', '24930577', '25024660', '25051268', '25061837', '25348125', '25424865', '25456277', '25581922', '25756280', '25759762', '25883640'],
'AR04': ['20955866', '21677894', '21826029', '22035386', '22215925', '22239924', '22294809', '22470524', '22959616', '23941878', '25142762', '25325584'],
'AR05': ['20049632', '20497902', '20661292', '20939652', '21034683', '21259270', '21376761', '21392554', '21440905', '21463060', '21477924', '21550590', '21623612', '21626350', '21898707', '22088577', '22285891', '22286850', '22315106', '22776995', '22911829', '24156344', '25036222', '25107317', '25749431']}

In [26]:
update_list = [pmid for subset in all_found_d.values() for pmid in subset]
len(update_list)

156

In [39]:
cmd = ' '.join(update_list)
print(cmd)
!medic update {cmd} 2> /dev/null
print()
!medic --format tsv write ALL 2> /dev/null | cut -f 1 | wc -l

20049632 20497902 20661292 20939652 21034683 21259270 21376761 21392554 21440905 21463060 21477924 21550590 21623612 21626350 21898707 22088577 22285891 22286850 22315106 22776995 22911829 24156344 25036222 25107317 25749431 20955866 21677894 21826029 22035386 22215925 22239924 22294809 22470524 22959616 23941878 25142762 25325584 22664396 20410871 20532489 21050743 21278378 21278907 22004270 22048839 22146934 22169884 22233352 22378876 22414937 22474609 22505867 22686386 22715197 22805501 22832959 22968207 23190433 23299717 23504052 23555220 23770566 23934417 24126129 24466064 24512610 24651580 24933663 24964082 25521352 20450941 20525011 20584721 20615239 20620104 20634711 20663220 20842164 20857862 21077571 21118712 21120131 21179552 21203376 21206465 21276977 21280045 21335015 21397252 21419826 21426626 21675365 21731598 21816115 21854953 21886801 21909371 21954087 22043127 22119526 22131608 22163262 22215928 22314045 22377810 22379239 22467988 22496862 22665872 22986355 22998925 2

229 pubmed ids in database

## write the new pmids to file batch_04_AR
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/task_data_temp/batch_04_AR


# Need new copies of:

'04_AR04_23941878.pdf'
'04_AR03_23193115.pdf'